In [1]:
import json
import requests
import pandas as pd
from collections import Counter
from bs4 import BeautifulSoup

/Users/deion/Lab-Link/.venv/lib/python3.8/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


# Get Organizations

### Import Nasa Project Data JSON

In [116]:
with open('./project_data.json', 'r') as f:
    raw_data_dict = json.load(f)
projects_arr = raw_data_dict

In [8]:
# projects_arr = raw_data_dict.get("_project", None)

### Get Unique Organizations (Sponsors)

In [51]:
org_dict={}
def fill_org_dict(sponsor, project):
    org_dict[sponsor] = project['project_url_external']
    return sponsor

all_orgs = [fill_org_dict(sponsor, project) for project in projects_arr if len(project['project_owners']) > 0 for sponsor in project['project_owners'].split(',')]

In [52]:
len(org_dict)

47

In [64]:
unique_orgs = org_dict.keys()

In [73]:
url_list = ['https://www.fisheries.noaa.gov/themes/custom/noaa_components/images/NOAA_FISHERIES_logoH.png', 'https://www.kenaiwatershed.org/wp-content/uploads/2017/10/kwf_logo.png', 'https://serc.si.edu/sites/all/themes/si_serc/logo-icon.png', 'https://www.dri.edu/wp-content/uploads/DRI-logo-taglineWEB2.png', 'https://www.fs.usda.gov/FSCSThemeStatic/themes/usda_nfs/img/fslogo.svg']

In [106]:
new_orgs = {}
for i, org in enumerate(org_dict.items()):
    pos = (i % len(url_list)) 
    url = url_list[pos] 
    new_orgs[org[0]] = url

In [107]:
org_obj_list = []
for org, org_url in new_orgs.items():
    org_obj = {"organization_name":org, "image_url":org_url, 'organization_description': '', "organization_milestones": ''}
    org_obj_list.append(org_obj)

In [109]:
with open('org_data.json', 'w') as f:
    json.dump(org_obj_list, f)

### Prepare Project Data to Meet Project Model Structure

In [111]:
projects_arr[0].keys()

dict_keys(['project_name', 'project_url_on_catalog', 'project_url_external', 'project_description', 'project_keywords', 'project_fields_of_science', 'project_is_active', 'project_owners', 'project_collaborators', 'project_geographical_scope', 'project_targeted_groups', 'projects_tasks', 'project_email', 'project_started_date'])

In [112]:
 key_mapping = {'project_name': 'project_name',
'project_url_on_catalog': 'project_url_on_catalog',
'project_url_external': 'project_url_external',
'project_description': 'project_description',
'keywords': 'project_keywords',
'fields_of_science': 'project_fields_of_science',
'project_status': 'project_is_active', 
'agency_sponsor': 'project_owners', 
'agency_sponsor_other': 'project_collaborators',
'geographic_scope': 'project_geographical_scope',
'participant_age': 'project_targeted_groups',
'participation_tasks': 'projects_tasks',
'email': 'project_email', 
'start_date': 'project_started_date', 
}
def clean_keys(obj):
    new_obj = {key_mapping.get(k, k): v for k, v in obj.items() if k in key_mapping.keys()}
    return new_obj
    
cleaned_projects_list = [clean_keys(project) for project in projects_arr]

In [118]:
cleaned_projects_list = projects_arr

In [119]:
def create_job(job, project):
    words = job.split('_')
    capitalized_words = [word.strip().capitalize() for word in words]
    result = ' '.join(capitalized_words)
    print(project['projects_tasks'])
    project['projects_tasks'] = result
    print(project['projects_tasks'])
    return project
    
seperated_projects_list = [create_job(job, project) for project in projects_arr if project.get('projects_tasks') for job in project.get('projects_tasks').split(',') if len(job) > 0]

audio_video_recording, identification, learning, measurement, observation, photography
Audio Video Recording
Audio Video Recording
Identification
Identification
Learning
Learning
Measurement
Measurement
Observation
Observation
Photography
identification
Identification
learning
Learning
measurement
Measurement
data_entry, identification, learning, measurement, observation, sample_analysis, site_selection_description, specimen_sample_collection
Data Entry
Data Entry
Identification
Identification
Learning
Learning
Measurement
Measurement
Observation
Observation
Sample Analysis
Sample Analysis
Site Selection Description
Site Selection Description
Specimen Sample Collection
geolocation, specimen_sample_collection
Geolocation
Geolocation
Specimen Sample Collection
classification_tagging, data_analysis
Classification Tagging
Classification Tagging
Data Analysis
data_analysis, data_entry, identification, learning, measurement, observation, photography
Data Analysis
Data Analysis
Data Entry
Dat

In [122]:
filtered_projects = list(filter(lambda x: len(x['projects_tasks']) > 0 ,projects_arr))

In [123]:
len(cleaned_projects_list)

506

In [124]:
len(filtered_projects)

486

In [127]:
with open('./seed_jsons/project_data.json', 'w') as f:
    json.dump(cleaned_projects_list, f)